# Automl

automl을 활용해 , 모델 선정

https://velog.io/@gyounghwan1002/python-AutoML%EB%9D%BC%EC%9D%B4%EB%B8%8C%EB%9F%AC%EB%A6%AC-pycaret-%EC%82%AC%EC%9A%A9%EB%B2%95


https://john-analyst.medium.com/pycaret-%EC%BD%94%EB%93%9C-%EB%AA%87-%EC%A4%84%EB%A1%9C-%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0-18a8de4b6024




In [1]:
#!pip install pycaret[full]
#!pip install scikit-learn==0.23.2 --user


In [2]:
import pycaret

In [3]:
import pandas as pd
import sklearn
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#전처리들

train = pd.read_csv('data/train.csv') 
test = pd.read_csv('data/test.csv')


train=train.drop(['id'],axis=1)
test=test.drop(['id'],axis=1)

#na 값을 시간별 평균으로 넣어주자.


def fill_na(df,column):
    print('column : ',column)
    hour_mean = train.groupby('hour').mean()[column]
    
    na_index_list = list( df[df[column].isnull()].index ) #na 인덱스를 받아서.
    
    for ind in na_index_list:
        df.loc[ind,column] = hour_mean[train.loc[ind,'hour']]
        if np.isnan( hour_mean[train.loc[ind,'hour']] ):
            df.loc[ind,column] = train[column].mean()
            #왜냐면, 진짜 그시간대는 모두 na인 경우가 있다. 그때는 전체 평균
             
fill_na(train,'hour_bef_temperature')
fill_na(test,'hour_bef_temperature')

fill_na(train,'hour_bef_windspeed')
fill_na(test,'hour_bef_windspeed')

fill_na(train,'hour_bef_humidity')
fill_na(test,'hour_bef_humidity')

fill_na(train,'hour_bef_visibility')
fill_na(test,'hour_bef_visibility')

fill_na(train,'hour_bef_ozone')
fill_na(test,'hour_bef_ozone')

fill_na(train,'hour_bef_pm10')
fill_na(test,'hour_bef_pm10')

fill_na(train,'hour_bef_pm2.5')
fill_na(test,'hour_bef_pm2.5')


#나머지 데이터 0으로 채우기
train.fillna(0,inplace = True)
test.fillna(0,inplace = True)





column :  hour_bef_temperature
column :  hour_bef_temperature
column :  hour_bef_windspeed
column :  hour_bef_windspeed
column :  hour_bef_humidity
column :  hour_bef_humidity
column :  hour_bef_visibility
column :  hour_bef_visibility
column :  hour_bef_ozone
column :  hour_bef_ozone
column :  hour_bef_pm10
column :  hour_bef_pm10
column :  hour_bef_pm2.5
column :  hour_bef_pm2.5


In [5]:
from pycaret.regression import *


#각종 전처리들을 수행
#feature 선택, scaling 등 모델에 맞게 실시해준다.
reg_test_1 = setup(data=train,
                   target='count', #label 변수를 넣는다.
                   train_size= 0.8,
                   fold=5)

,Description,Value
0,session_id,3451
1,Target,count
2,Original Data,"(1459, 10)"
3,Missing Values,False
4,Numeric Features,8
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(1167, 9)"


In [6]:
compare_models()
#validation set에서 가장 우수했던 모델들을 보여준다.

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,24.9143,1368.3685,36.8516,0.7937,0.5250,0.7015,0.6020
et,Extra Trees Regressor,24.6003,1365.4480,36.8783,0.7935,0.4903,0.6368,0.0820
rf,Random Forest Regressor,25.7075,1449.5247,37.9964,0.7808,0.5089,0.7411,0.1000
xgboost,Extreme Gradient Boosting,26.0330,1500.8901,38.6411,0.7734,0.5235,0.6873,0.1040
gbr,Gradient Boosting Regressor,26.6433,1551.6490,39.2599,0.7658,0.5270,0.6921,0.0340
lightgbm,Light Gradient Boosting Machine,26.5214,1557.6577,39.3904,0.7642,0.5709,0.8435,0.0300
ada,AdaBoost Regressor,41.3088,2535.8066,50.3287,0.6156,0.8410,1.7774,0.0300
lr,Linear Regression,38.9566,2653.6069,51.4114,0.5999,0.7872,1.2593,0.9480
ridge,Ridge Regression,39.0032,2661.7456,51.4955,0.5987,0.8010,1.2613,0.0040
lar,Least Angle Regression,39.1046,2663.7564,51.5116,0.5984,0.7896,1.2502,0.0060


In [7]:
et = create_model('et')
#구체적 성능확인

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,25.3354,1585.1337,39.8137,0.7705,0.4925,0.6742
1,25.0327,1246.0852,35.2999,0.8015,0.5154,0.6959
2,23.8774,1213.0756,34.8292,0.8008,0.4873,0.5569
3,25.5852,1571.2126,39.6385,0.7669,0.4036,0.3918
4,23.1709,1211.7330,34.8100,0.8277,0.5528,0.8653
Mean,24.6003,1365.4480,36.8783,0.7935,0.4903,0.6368
Std,0.9234,174.1804,2.3325,0.0225,0.0491,0.1572


In [17]:
cb = create_model('catboost')
#구체적 성능확인

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,30.0680,1711.9832,41.3761,0.7657,0.5526,0.8193
1,23.7795,1163.5182,34.1104,0.8133,0.6029,0.8018
2,23.3946,1052.1835,32.4374,0.8438,0.5001,0.5245
3,25.7986,1731.2779,41.6086,0.7419,0.5733,1.6002
4,27.8929,1681.2022,41.0025,0.7353,0.6009,0.8641
Mean,26.1867,1468.0330,38.1070,0.7800,0.5660,0.9220
Std,2.5186,296.6179,3.9862,0.0420,0.0378,0.3595


# 파라미터 튜닝

## extra tree

In [12]:
et = create_model('et')
et = tune_model(et,
                optimize='RMSE',
                n_iter=100,)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,26.2416,1683.2122,41.0270,0.7563,0.4987,0.6742
1,25.0513,1293.7628,35.9689,0.7939,0.5280,0.7335
2,24.1745,1212.2539,34.8174,0.8009,0.4771,0.5362
3,26.3059,1717.0608,41.4374,0.7453,0.4195,0.4058
4,23.9893,1305.6871,36.1343,0.8144,0.5805,1.0739
Mean,25.1525,1442.3954,37.8770,0.7821,0.5008,0.6847
Std,0.9835,213.1571,2.7798,0.0267,0.0534,0.2253


In [19]:
#튜닝 결과
print(et)

ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                    max_depth=11, max_features=1.0, max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0,
                    min_impurity_split=None, min_samples_leaf=2,
                    min_samples_split=7, min_weight_fraction_leaf=0.0,
                    n_estimators=190, n_jobs=-1, oob_score=False,
                    random_state=3451, verbose=0, warm_start=False)


In [20]:
# test 데이터 추론

final_model = finalize_model(et)
prediction = predict_model(final_model, data=test)


In [21]:
prediction

,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,Label
0,7,20.7,0.0,1.3,62.0,954.0,0.041000,44.000000,27.000000,101.613158
1,17,30.0,0.0,5.4,33.0,1590.0,0.061000,49.000000,36.000000,222.200000
2,13,19.0,1.0,2.1,95.0,193.0,0.020000,36.000000,28.000000,50.878947
3,6,22.5,0.0,2.5,60.0,1185.0,0.027000,52.000000,38.000000,45.915789
4,22,14.6,1.0,3.4,93.0,218.0,0.041000,18.000000,15.000000,52.515789
...,...,...,...,...,...,...,...,...,...,...
710,1,24.6,0.0,2.4,60.0,1745.0,0.035148,54.426230,30.593220,68.623684
711,1,18.1,0.0,1.0,55.0,2000.0,0.053344,58.283333,29.844828,61.626316
712,9,23.3,0.0,2.3,66.0,1789.0,0.020000,17.000000,15.000000,116.865789
713,16,27.0,0.0,1.6,46.0,1956.0,0.032000,40.000000,26.000000,157.778947


In [22]:
submission = pd.read_csv('./data/submission.csv')
submission

,id,count
0,0,1
1,1,1
2,2,1
3,4,1
4,5,1
...,...,...
710,2148,1
711,2149,1
712,2165,1
713,2166,1


In [23]:
submission['count'] = prediction['Label']

In [24]:
submission.to_csv('베이스라인.csv',index = False)

## catboost tuning

In [27]:
cat = create_model('catboost')
cat = tune_model(cat,
                optimize='RMSE',
                n_iter=100,)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,26.5878,1746.4029,41.7900,0.7472,0.5105,0.7054
1,25.8749,1241.8035,35.2392,0.8022,0.5653,0.9550
2,23.6447,1108.8541,33.2995,0.8179,0.4508,0.4916
3,26.4568,1613.7866,40.1720,0.7606,0.4830,0.4590
4,24.7664,1320.9705,36.3452,0.8122,0.6339,1.3916
Mean,25.4661,1406.3635,37.3692,0.7880,0.5287,0.8005
Std,1.1148,237.2909,3.1479,0.0286,0.0646,0.3448


In [32]:
#튜닝 결과
print(cat.get_all_params())

{'nan_mode': 'Min', 'eval_metric': 'RMSE', 'iterations': 190, 'sampling_frequency': 'PerTree', 'leaf_estimation_method': 'Newton', 'grow_policy': 'SymmetricTree', 'penalties_coefficient': 1, 'boosting_type': 'Plain', 'model_shrink_mode': 'Constant', 'feature_border_type': 'GreedyLogSum', 'bayesian_matrix_reg': 0.10000000149011612, 'eval_fraction': 0, 'force_unit_auto_pair_weights': False, 'l2_leaf_reg': 9, 'random_strength': 0.20000000298023224, 'rsm': 1, 'boost_from_average': True, 'model_size_reg': 0.5, 'pool_metainfo_options': {'tags': {}}, 'subsample': 0.800000011920929, 'use_best_model': False, 'random_seed': 3451, 'depth': 8, 'posterior_sampling': False, 'border_count': 254, 'classes_count': 0, 'auto_class_weights': 'None', 'sparse_features_conflict_fraction': 0, 'leaf_estimation_backtracking': 'AnyImprovement', 'best_model_min_trees': 1, 'model_shrink_rate': 0, 'min_data_in_leaf': 1, 'loss_function': 'RMSE', 'learning_rate': 0.15000000596046448, 'score_function': 'Cosine', 'task

In [33]:
# test 데이터 추론

final_model = finalize_model(cat)
prediction = predict_model(final_model, data=test)


In [34]:
prediction

,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,Label
0,7,20.7,0.0,1.3,62.0,954.0,0.041000,44.000000,27.000000,104.137317
1,17,30.0,0.0,5.4,33.0,1590.0,0.061000,49.000000,36.000000,195.894932
2,13,19.0,1.0,2.1,95.0,193.0,0.020000,36.000000,28.000000,57.092030
3,6,22.5,0.0,2.5,60.0,1185.0,0.027000,52.000000,38.000000,69.115402
4,22,14.6,1.0,3.4,93.0,218.0,0.041000,18.000000,15.000000,67.264402
...,...,...,...,...,...,...,...,...,...,...
710,1,24.6,0.0,2.4,60.0,1745.0,0.035148,54.426230,30.593220,72.302246
711,1,18.1,0.0,1.0,55.0,2000.0,0.053344,58.283333,29.844828,70.645147
712,9,23.3,0.0,2.3,66.0,1789.0,0.020000,17.000000,15.000000,119.346160
713,16,27.0,0.0,1.6,46.0,1956.0,0.032000,40.000000,26.000000,130.847717


In [35]:
submission = pd.read_csv('./data/submission.csv')
submission

,id,count
0,0,1
1,1,1
2,2,1
3,4,1
4,5,1
...,...,...
710,2148,1
711,2149,1
712,2165,1
713,2166,1


In [36]:
submission['count'] = prediction['Label']

In [37]:
submission.to_csv('베이스라인.csv',index = False)

## lightgbm

In [38]:
lgbm = create_model('lightgbm')
lgbm = tune_model(lgbm,
                optimize='RMSE',
                n_iter=100,)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,27.4663,1678.7066,40.9720,0.7570,0.5566,0.8537
1,26.4570,1310.8789,36.2061,0.7912,0.5811,0.9333
2,25.5442,1257.5663,35.4622,0.7934,0.5216,0.6485
3,27.8898,1725.1921,41.5354,0.7440,0.5115,0.5776
4,24.7527,1342.8228,36.6445,0.8091,0.6435,1.3643
Mean,26.4220,1463.0334,38.1640,0.7789,0.5628,0.8755
Std,1.1665,197.5148,2.5571,0.0244,0.0474,0.2768


In [39]:
#튜닝 결과
print(lgbm)

LGBMRegressor(bagging_fraction=0.6, bagging_freq=2, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
              importance_type='split', learning_rate=0.01, max_depth=-1,
              min_child_samples=6, min_child_weight=0.001, min_split_gain=0.5,
              n_estimators=250, n_jobs=-1, num_leaves=80, objective=None,
              random_state=3451, reg_alpha=0.3, reg_lambda=0.005, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


In [45]:
# test 데이터 추론

final_model = finalize_model(lgbm)
prediction = predict_model(final_model, data=test)


In [47]:
submission = pd.read_csv('./data/submission.csv')
submission

,id,count
0,0,1
1,1,1
2,2,1
3,4,1
4,5,1
...,...,...
710,2148,1
711,2149,1
712,2165,1
713,2166,1


In [48]:
submission['count'] = prediction['Label']

In [49]:
submission.to_csv('베이스라인.csv',index = False)

## ensemble

In [50]:
#lightgbm = create_model('lightgbm')
#catboost = create_model('catboost')

blended = blend_models(estimator_list = [et,lgbm, cat], fold = 5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,26.4449,1686.0273,41.0613,0.7559,0.5105,0.7379
1,25.5753,1230.1137,35.0730,0.8040,0.5769,0.9296
2,23.8221,1122.9635,33.5106,0.8156,0.4620,0.5286
3,26.4225,1617.0631,40.2127,0.7601,0.4673,0.4839
4,24.1492,1294.2316,35.9754,0.8160,0.6265,1.3670
Mean,25.2828,1390.0799,37.1666,0.7903,0.5286,0.8094
Std,1.1094,221.4649,2.9535,0.0268,0.0639,0.3210


In [51]:
# test 데이터 추론

final_model = finalize_model(blended)
prediction = predict_model(final_model, data=test)


In [52]:
submission = pd.read_csv('./data/submission.csv')
submission

,id,count
0,0,1
1,1,1
2,2,1
3,4,1
4,5,1
...,...,...
710,2148,1
711,2149,1
712,2165,1
713,2166,1


In [53]:
submission['count'] = prediction['Label']

In [55]:
submission.to_csv('베이스라인.csv',index = False)